# Georreferenciamento e análise de informações do banco de sentenças do TJSP

### José Eduardo de Souza Pimentel - 2026

In [1]:
# Importações necessárias

import os
import json
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Carrega variáveis de ambiente

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [3]:
# Configurações

MODELO = "gpt-5-nano" #"gpt-5-mini"
PATH_SENTENCAS = "dados"
MAX_INTERACOES = 3
CSV_SENTENCAS = 'dados\\sentencas.csv'
CSV_SENTENCAS_ANALISADAS = 'dados\\sentencas_analisadas.csv'
CSV_SENTENCAS_COM_ENDERECOS = 'dados\\sentencas_com_enderecos.csv'
CSV_SENTENCAS_GEORREFERENCIADAS = 'dados\\sentencas_georreferenciadas.csv'

In [4]:
# Parametrização da requisição ao TJSP

PARAMS_TJSP = params = {
    "conversationId": "",
    "dadosConsulta.pesquisaLivre": "",
    "tipoNumero": "UNIFICADO",
    "numeroDigitoAnoUnificado": "",
    "foroNumeroUnificado": "",
    "dadosConsulta.nuProcesso": "",
    "dadosConsulta.nuProcessoAntigo": "",
    "classeTreeSelection.values": "",
    "classeTreeSelection.text": "",
    "assuntoTreeSelection.values": "3608",
    "assuntoTreeSelection.text": "Tráfico de Drogas e Condutas Afins",
    "agenteSelectedEntitiesList": "",
    "contadoragente": "0",
    "contadorMaioragente": "0",
    "cdAgente": "",
    "nmAgente": "",
    "dadosConsulta.dtInicio": "",
    "dadosConsulta.dtFim": "",
    "varasTreeSelection.values": "320-7016,320-3967,320-7012,320-5,320-10,320-5637,320-6,320-3940,320-5712",
    "varasTreeSelection.text": "9 Registros selecionados",
    "dadosConsulta.ordenacao": "DESC"
}

# Download das sentenças

In [5]:
# Cria uma sessão persistente
session = requests.Session()

# 1a requisição para obter os cookies
response = session.get("https://esaj.tjsp.jus.br/cjpg/pesquisar.do", params=PARAMS_TJSP)

if response.status_code == 200:
    for i in range(1, MAX_INTERACOES + 1):
        pagina = str(i)
        url_pagina = f"https://esaj.tjsp.jus.br/cjpg/trocarDePagina.do?pagina={pagina}&conversationId="

        # Usa a mesma sessão para fazer a requisição
        response = session.get(url_pagina)

        if response.status_code == 200:
            # Salva o conteúdo da página
            with open(f"{PATH_SENTENCAS}/pagina_{pagina}.html", "wb") as file:
                file.write(response.content)
            print(f"Página {pagina} salva com sucesso.")
        else:
            print(f"Erro ao acessar a página {pagina}: {response.status_code}")

        # Intervalo entre requisições
        time.sleep(1)
else:
    print("Erro ao acessar a página inicial.")

Página 1 salva com sucesso.
Página 2 salva com sucesso.
Página 3 salva com sucesso.


# Criação do dataframe

In [6]:
def extrair_informacoes_html(html_path):
    processos = []
    with open(html_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
        textos = soup.find_all(class_="fundocinza1")
        for texto in textos:
            processo = {}
            # Número do Processo
            processo['numero_do_processo'] = texto.find('span', class_='fonteNegrito').text.strip()

            # Classe, Assunto, Magistrado, Comarca, Foro, Vara, Data de Disponibilização
            for row in texto.find_all('tr', class_='fonte'):
                if row.find('strong'):
                    key = row.find('strong').text.strip().replace(':', '').lower().replace(' ', '_')
                    value = row.find('strong').next_sibling.strip()
                    processo[key] = value

            # Sentença
            sentenca_divs = texto.find_all('div', align='justify')
            sentenca = ''
            for div in sentenca_divs:
                span = div.find('span')
                if span:
                    sentenca += span.text.strip() + '\n'
            processo['sentenca'] = sentenca.strip()

            processos.append(processo)

        return processos

In [7]:
data = []

for filename in os.listdir(PATH_SENTENCAS):
    if filename.endswith(".html"):
        filepath = os.path.join(PATH_SENTENCAS, filename)
        data += (extrair_informacoes_html(filepath))

# Cria o DataFrame
df = pd.DataFrame(data)
df.head(5)

,numero_do_processo,classe,assunto,magistrado,comarca,foro,vara,data_de_disponibilização,sentenca
0,1500535-45.2023.8.26.0551,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,GUILHERME LOPES ALVES LAMAS,Limeira,Foro de Limeira,2ª Vara Criminal,06/11/2025,SENTENÇA\n\nProcesso Digital nº:\t1500535-45.2...
1,1500514-35.2024.8.26.0551,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,NaN,Limeira,Foro de Limeira,3ª Vara Criminal,06/11/2025,
2,1503596-54.2025.8.26.0320,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Rudi Hiroshi Shinen,Limeira,Foro de Limeira,3ª Vara Criminal,05/11/2025,SENTENÇA\n\nProcesso Digital nº:\t1503596-54.2...
3,1500290-63.2025.8.26.0551,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Rudi Hiroshi Shinen,Limeira,Foro de Limeira,3ª Vara Criminal,05/11/2025,SENTENÇA\n\nProcesso Digital nº:\t1500290-63.2...
4,1503322-90.2025.8.26.0320,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Rudi Hiroshi Shinen,Limeira,Foro de Limeira,3ª Vara Criminal,05/11/2025,SENTENÇA\n\nProcesso Digital nº:\t1503322-90.2...


In [8]:
# Cria o dataframe com sentenças não vazias e salva em CSV
df_1 = df[(df['sentenca']!='')]
df_1 = df_1.drop_duplicates(subset=['numero_do_processo'], keep='last') # Despreza os processos repetidos
df_1.reset_index(drop=True, inplace=True)
df_1.to_csv(CSV_SENTENCAS, index=False, encoding='utf-8-sig')  # utf-8-sig para compatibilidade com Excel
print(f"Arquivo CSV salvo em: {CSV_SENTENCAS}")

Arquivo CSV salvo em: dados\sentencas.csv


# Consulta à API OpenAI para análise das sentenças

In [9]:
SYSTEM_MSG = (
    "Você é assessor jurídico especializado em direito penal. "
    "Responda exclusivamente em JSON válido, sem comentários, com as chaves: "
    "'data_fato', 'quantidade_reus', 'endereco_fato', 'responsavel_prisao', "
    "'modus_operandi_reu', 'alegacao_reu', 'resultado_processo', 'resumo_sentenca'."
)

USER_TEMPLATE = (
    "Analise a sentença a seguir e devolva um objeto JSON contendo as chaves já indicadas."
    "Para a chave 'data_fato' responda apenas com a data do fato (ex.: 24/03/2025)"
    "ou escreva 'Não se aplica'."
    "Para a chave 'quantidade_reus' responda apenas com um número (ex.: 1, 2, 3 etc)"
    "ou escreva 'Não se aplica'."
    "Para a chave 'endereco_fato' responda com o endereço principal do fato "
    "(ex.: rua Alvorada, 123, bairro Jardim Brasília, Piracicaba/SP)."
    "ou escreva 'Não se aplica'."
    "Para a chave 'responsavel_prisao', informe que tipo de agente público realizou a prisão "
    "(ex.: policial militar, policial civil, policial penal, guarda municipal, outros) "
    "ou escreva 'Não se aplica'."
    "Para a chave 'modus_operandi_reu', resuma em até 70 palavras de que forma o tráfico era realizado "
    "ou escreva 'Não se aplica'."
    "Para a chave 'alegacao_reu', resuma em até 70 palavras o que o réu ou os réus alegaram em suas defesas "
    "ou escreva 'Não se aplica'."
    "Para a chave 'resultado_processo', responda 'Procedente', 'Improcedente', 'Parcialmente procedente' "
    "ou escreva 'Não se aplica'."
    "Para a chave 'resumo_sentenca', resuma em até 100 palavras a sentença em sua totalidade."
    "RESTRIÇÕES: para todos os casos, não se refira ao(s) réu(s) pelo nome. Use suas iniciais.\n\n"
    "SENTENÇA:\n\n{sentenca}"
)


client = OpenAI()

def analisar_sentenca_openai(sentenca: str) -> dict:
    """
    Analisa o texto de uma sentença usando modelo da OpenAI
    e retorna os dados extraídos em formato de dicionário Python.
    """

    resposta_api = None

    try:
        response = client.chat.completions.create(
            model=MODELO,
            response_format={"type": "json_object"},  # força o JSON
            messages=[
                {"role": "system", "content": SYSTEM_MSG},
                {"role": "user", "content": USER_TEMPLATE.format(sentenca=sentenca)}
            ]
        )

        if (
            response
            and response.choices
            and response.choices[0].message
            and response.choices[0].message.content
        ):
            resposta_api = response.choices[0].message.content.strip()
        else:
            resposta_api = "Erro: Resposta da API em formato inesperado."
            print(f"Resposta inesperada da API: {response}")

    except Exception as e:
        resposta_api = f"Erro ao processar com OpenAI: {e}"

    return resposta_api

In [10]:
lista_de_respostas = []
for idx, row in df_1.iterrows():
    resposta_texto = analisar_sentenca_openai(row['sentenca'])  # string retornada pela API
    try:
        analise = json.loads(resposta_texto)  # converte a string JSON para dict
    except Exception as e:
        print(f"Erro ao converter JSON no processo {row['numero_do_processo']}: {e}")
        print("Resposta bruta da API:", resposta_texto)
        # fallback: cria dicionário com valores padrão para não quebrar o pipeline
        analise = {
            "data_fato": "Não se aplica",
            "quantidade_reus": "Não se aplica",
            "endereco_fato": "Não se aplica",
            "responsavel_prisao": "Não se aplica",
            "modus_operandi_reu": "Não se aplica",
            "alegacao_reu": "Não se aplica",
            "resultado_processo": "Não se aplica",
            "resumo_sentenca": "Não se aplica"
        }
    n_processo = row['numero_do_processo']
    print(f"Analisando o processo {n_processo}...")
    lista_de_respostas.append((n_processo, analise))

Analisando o processo 1500535-45.2023.8.26.0551...
Analisando o processo 1503596-54.2025.8.26.0320...
Analisando o processo 1500290-63.2025.8.26.0551...
Analisando o processo 1503322-90.2025.8.26.0320...
Analisando o processo 1502110-34.2025.8.26.0320...
Analisando o processo 1503640-73.2025.8.26.0320...
Analisando o processo 1503776-70.2025.8.26.0320...
Analisando o processo 1502698-41.2025.8.26.0320...
Analisando o processo 1503732-51.2025.8.26.0320...
Analisando o processo 1503517-75.2025.8.26.0320...
Analisando o processo 1500625-82.2025.8.26.0551...
Analisando o processo 1501493-74.2025.8.26.0320...
Analisando o processo 1504095-38.2025.8.26.0320...
Analisando o processo 1503596-88.2024.8.26.0320...
Analisando o processo 1501311-88.2025.8.26.0320...


In [11]:
# Cria um DataFrame a partir da lista de respostas
# A lista_de_respostas contém tuplas (numero_do_processo, dict_analise)
# Vamos expandir o dicionário em colunas do DataFrame

data_para_df = []
for numero_processo, analise_dict in lista_de_respostas:
  row_data = {'numero_do_processo': numero_processo}
  row_data.update(analise_dict) # Adiciona as chaves do dicionário como colunas
  data_para_df.append(row_data)

In [12]:
df_respostas = pd.DataFrame(data_para_df)
display(df_respostas.head(5))

df_respostas.to_csv(CSV_SENTENCAS_ANALISADAS, index=False, encoding='utf-8-sig')
print(f"Arquivo CSV salvo em: {CSV_SENTENCAS_ANALISADAS}")

,numero_do_processo,data_fato,quantidade_reus,endereco_fato,responsavel_prisao,modus_operandi_reu,alegacao_reu,resultado_processo,resumo_sentenca
0,1500535-45.2023.8.26.0551,14/09/2023,1,"Rua Quatro, Odécio Degan, Limeira/SP",guarda municipal,"Durante patrulhamento, guardas avistaram o réu...","A defesa alegou absolvição; subsidiariamente, ...",Procedente,JVRG foi julgado procedente pela prática do de...
1,1503596-54.2025.8.26.0320,18/08/2025,2,"Rua Artur Reis, 550, Jardim Nova Suíça, Limeir...",policiais militares,"Os indiciados foram abordados em área de mata,...",J.W.S.G. e D.C.B. alegaram ser apenas usuários...,Procedente,"Sentença condenatória proferida em Limeira, 05..."
2,1500290-63.2025.8.26.0551,25/04/2025,1,"Rua Moacir Barros Mognani, 100, Recanto dos Pá...",policiais militares,"Durante diligência em ponto de tráfico, o réu ...","Negou a prática delitiva, alegando ser usuário...",Procedente,O juízo reconheceu a materialidade e a autoria...
3,1503322-90.2025.8.26.0320,04/08/2025,1,"Rua João Maesi, 141, Jardim Colina Verde, Lime...",guarda municipal,K.V.O. atuava em ponto de venda de drogas próx...,K.V.O. alegou possuir apenas as porções encont...,Procedente,Condeno K.V.O. à pena de 5 anos e 10 meses de ...
4,1502110-34.2025.8.26.0320,30/04/2025,1,"Rua João Ciarrochi, 250, Jardim Olga Veroni, L...",policial militar,O réu foi visto retirando entorpecentes de uma...,Negou as acusações. Alegou não ter envolviment...,Procedente,"Condenação por tráfico de drogas (art. 33, §4º..."


Arquivo CSV salvo em: dados\sentencas_analisadas.csv


In [13]:
df_geral = pd.merge(df_1, df_respostas, on='numero_do_processo')
df_geral.head(5)

,numero_do_processo,classe,assunto,magistrado,comarca,foro,vara,data_de_disponibilização,sentenca,data_fato,quantidade_reus,endereco_fato,responsavel_prisao,modus_operandi_reu,alegacao_reu,resultado_processo,resumo_sentenca
0,1500535-45.2023.8.26.0551,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,GUILHERME LOPES ALVES LAMAS,Limeira,Foro de Limeira,2ª Vara Criminal,06/11/2025,SENTENÇA\n\nProcesso Digital nº:\t1500535-45.2...,14/09/2023,1,"Rua Quatro, Odécio Degan, Limeira/SP",guarda municipal,"Durante patrulhamento, guardas avistaram o réu...","A defesa alegou absolvição; subsidiariamente, ...",Procedente,JVRG foi julgado procedente pela prática do de...
1,1503596-54.2025.8.26.0320,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Rudi Hiroshi Shinen,Limeira,Foro de Limeira,3ª Vara Criminal,05/11/2025,SENTENÇA\n\nProcesso Digital nº:\t1503596-54.2...,18/08/2025,2,"Rua Artur Reis, 550, Jardim Nova Suíça, Limeir...",policiais militares,"Os indiciados foram abordados em área de mata,...",J.W.S.G. e D.C.B. alegaram ser apenas usuários...,Procedente,"Sentença condenatória proferida em Limeira, 05..."
2,1500290-63.2025.8.26.0551,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Rudi Hiroshi Shinen,Limeira,Foro de Limeira,3ª Vara Criminal,05/11/2025,SENTENÇA\n\nProcesso Digital nº:\t1500290-63.2...,25/04/2025,1,"Rua Moacir Barros Mognani, 100, Recanto dos Pá...",policiais militares,"Durante diligência em ponto de tráfico, o réu ...","Negou a prática delitiva, alegando ser usuário...",Procedente,O juízo reconheceu a materialidade e a autoria...
3,1503322-90.2025.8.26.0320,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Rudi Hiroshi Shinen,Limeira,Foro de Limeira,3ª Vara Criminal,05/11/2025,SENTENÇA\n\nProcesso Digital nº:\t1503322-90.2...,04/08/2025,1,"Rua João Maesi, 141, Jardim Colina Verde, Lime...",guarda municipal,K.V.O. atuava em ponto de venda de drogas próx...,K.V.O. alegou possuir apenas as porções encont...,Procedente,Condeno K.V.O. à pena de 5 anos e 10 meses de ...
4,1502110-34.2025.8.26.0320,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,GUILHERME LOPES ALVES LAMAS,Limeira,Foro de Limeira,2ª Vara Criminal,04/11/2025,SENTENÇA\n\nProcesso Digital nº:\t1502110-34.2...,30/04/2025,1,"Rua João Ciarrochi, 250, Jardim Olga Veroni, L...",policial militar,O réu foi visto retirando entorpecentes de uma...,Negou as acusações. Alegou não ter envolviment...,Procedente,"Condenação por tráfico de drogas (art. 33, §4º..."


In [14]:
df_geral_com_endereco = df_geral[df_geral['endereco_fato'] != 'Não se aplica']
df_geral_com_endereco.head(5)

df_geral_com_endereco.to_csv(CSV_SENTENCAS_COM_ENDERECOS, index=False, encoding='utf-8-sig')
print(f"Arquivo CSV salvo em: {CSV_SENTENCAS_COM_ENDERECOS}")

Arquivo CSV salvo em: dados\sentencas_com_enderecos.csv


# Georreferenciamento do dataframe

In [15]:
def buscar_coords(endereco, api_key):
    """Função auxiliar para retornar latitude e longitude ou None em caso de erro."""
    if not endereco:
        return None, None

    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": endereco, "key": api_key, "language": "pt-BR"}

    try:
        response = requests.get(url, params=params)
        data = response.json()

        if data.get("status") == "OK":
            location = data["results"][0]["geometry"]["location"]
            return location["lat"], location["lng"]
        else:
            return None, None
    except:
        return None, None

In [16]:
df_geral_com_endereco[['latitude', 'longitude']] = df_geral_com_endereco['endereco_fato'].apply(
    lambda x: pd.Series(buscar_coords(x, GOOGLE_API_KEY))
)

In [17]:
df_geral_com_endereco.head(5)

,numero_do_processo,classe,assunto,magistrado,comarca,foro,vara,data_de_disponibilização,sentenca,data_fato,quantidade_reus,endereco_fato,responsavel_prisao,modus_operandi_reu,alegacao_reu,resultado_processo,resumo_sentenca,latitude,longitude
0,1500535-45.2023.8.26.0551,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,GUILHERME LOPES ALVES LAMAS,Limeira,Foro de Limeira,2ª Vara Criminal,06/11/2025,SENTENÇA\n\nProcesso Digital nº:\t1500535-45.2...,14/09/2023,1,"Rua Quatro, Odécio Degan, Limeira/SP",guarda municipal,"Durante patrulhamento, guardas avistaram o réu...","A defesa alegou absolvição; subsidiariamente, ...",Procedente,JVRG foi julgado procedente pela prática do de...,-22.614202,-47.412994
1,1503596-54.2025.8.26.0320,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Rudi Hiroshi Shinen,Limeira,Foro de Limeira,3ª Vara Criminal,05/11/2025,SENTENÇA\n\nProcesso Digital nº:\t1503596-54.2...,18/08/2025,2,"Rua Artur Reis, 550, Jardim Nova Suíça, Limeir...",policiais militares,"Os indiciados foram abordados em área de mata,...",J.W.S.G. e D.C.B. alegaram ser apenas usuários...,Procedente,"Sentença condenatória proferida em Limeira, 05...",-22.575703,-47.384675
2,1500290-63.2025.8.26.0551,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Rudi Hiroshi Shinen,Limeira,Foro de Limeira,3ª Vara Criminal,05/11/2025,SENTENÇA\n\nProcesso Digital nº:\t1500290-63.2...,25/04/2025,1,"Rua Moacir Barros Mognani, 100, Recanto dos Pá...",policiais militares,"Durante diligência em ponto de tráfico, o réu ...","Negou a prática delitiva, alegando ser usuário...",Procedente,O juízo reconheceu a materialidade e a autoria...,-22.616410,-47.416046
3,1503322-90.2025.8.26.0320,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Rudi Hiroshi Shinen,Limeira,Foro de Limeira,3ª Vara Criminal,05/11/2025,SENTENÇA\n\nProcesso Digital nº:\t1503322-90.2...,04/08/2025,1,"Rua João Maesi, 141, Jardim Colina Verde, Lime...",guarda municipal,K.V.O. atuava em ponto de venda de drogas próx...,K.V.O. alegou possuir apenas as porções encont...,Procedente,Condeno K.V.O. à pena de 5 anos e 10 meses de ...,-22.592034,-47.434266
4,1502110-34.2025.8.26.0320,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,GUILHERME LOPES ALVES LAMAS,Limeira,Foro de Limeira,2ª Vara Criminal,04/11/2025,SENTENÇA\n\nProcesso Digital nº:\t1502110-34.2...,30/04/2025,1,"Rua João Ciarrochi, 250, Jardim Olga Veroni, L...",policial militar,O réu foi visto retirando entorpecentes de uma...,Negou as acusações. Alegou não ter envolviment...,Procedente,"Condenação por tráfico de drogas (art. 33, §4º...",-22.583876,-47.374098


In [19]:
# Salvar o DataFrame georreferenciado em um novo arquivo CSV
df_geral_com_endereco.to_csv(CSV_SENTENCAS_GEORREFERENCIADAS, index=False, encoding='utf-8-sig')
print(f"DataFrame de respostas salvo em: {CSV_SENTENCAS_GEORREFERENCIADAS}")

DataFrame de respostas salvo em: dados\sentencas_georreferenciadas.csv


In [21]:
# Gera o arquivo para a visualização
df_geral_com_endereco[['numero_do_processo', 'data_fato', 'responsavel_prisao', 'modus_operandi_reu', 
                       'alegacao_reu', 'resultado_processo', 'resumo_sentenca', 'latitude', 'longitude']].to_csv(
    'dados.csv', index=False, encoding='utf-8-sig')